# (공부) Datamining HW
> 두번째 과제, Classification
- toc:true
- branch: master
- badges: true
- comments: true
- author: 김하영
- categories: [Study]

In [1]:
setwd("/home/khy/Desktop/2022STAT/datamining")

library(tidyverse)
library(dplyr)
library(gridExtra)
library(ggplot2)
library(tidyr)

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine




**1. 주식에 대한 배당금을 지급할지 여부(’Yes’, ’No’)를 예측하기 위해, 작년 한 해 동안의 수익률을 설명변수 X로 사용하고자 한다. 많은 수의 회사를 조사해보니 배당금을 지급한 회사의 주식 수익률의 평균 X¯=10 이고, 지급하지 않은 회사의 주식 수익률의 평균 X¯=0이다. 두 그룹에서
수익률의 분산은 두 그룹 모두 σˆ2=36이다. X가 정규분포를 따른다고 가정하자. 어떤 회사의 작년 수익률 X = 4였다면, 이 회사가 배당금을 지급할 확률은 얼마인가?**

**2. ‘Auto.csv’ 데이터를 이용하여 자동차의 연비가 높을지 낮을지에 대해 예측을 하고자 한다. 다음 물음에 답하여라. (NA 처리 후 분석)**

(a) mpg01 변수를 생성하여라. 이 변수는 mpg가 mpg의 중앙값보다 크면 1의 값을 갖고 아니면 0의 값을 갖는 변수이다. (중앙값 : median())

(b) 시각화를 통하여 mpg01과 다른 변수들 사이의 관계를 확인하고 설명하여라. 어떤 변수가 mp01을 예측하는 데 가장 유용할 것으로 생각되는가?

(c) 데이터를 training data (60%)와 testing data(40%)로 나누어라.

(d) (b)에서 연관이 있다고 생각되는 변수들을 이용하여, mpg01을 예측하기 위한 LDA를 수행하여라. test 오분류율은 얼마인가?

(e) (b)에서 연관이 있다고 생각되는 변수들을 이용하여, mpg01을 예측하기 위한 QDA를 수행하여라. test 오분류율은 얼마인가?

(f) (b)에서 연관이 있다고 생각되는 변수들을 이용하여, mpg01을 예측하기 위한 logistic regressiond을 수행하여라. test 오분류율은 얼마인가?

(g) (b)에서 연관이 있다고 생각되는 변수들을 이용하여, mpg01을 예측하기 위한 KNN을 수행
하여라. KNN을 수행할 때 몇개의 k 값을 선택하여 분석하여라. test 오분류율은 얼마인가?
어떤 k를 선택했을 때 결과가 가장 좋았는가?

**3. ‘Boston.csv’ 데이터를 이용하여, 어떤 지역의 범죄율이 중앙값 이상인지 아닌지를 예측하기 위한
분류 모형(logistic regression, LDA, KNN)을 적합하여라.**